In [1]:
using WordNet
using WordEmbeddings, SoftmaxClassifier
using Utils
using Query
using Distances
using Iterators
using NearestNeighbors
using JLD
using Trees
using AbstractTrees

In [2]:
ee = load("../eval/models/plain/tokenised_lowercase_WestburyLab.wikicorp.201004_50__i1.jld","ee");
#ee = restore("../eval/models/ss/keep/tokenised_lowercase_WestburyLab.wikicorp.201004_100_i1.model")
#dtree,labels = nn_using AbstractTreestree(ee)
#""

Word embedding(dimension = 50)of 88262 words


In [3]:
using Lumberjack
remove_truck("console")
add_truck(LumberjackTruck(STDOUT, "info", Dict{Symbol,Any}(:is_colorized => true)), "console")
add_truck(LumberjackTruck("semhuff_ipynb.log"), "file-logger")

Lumberjack.LumberjackTruck(IOStream(<file semhuff_ipynb.log>),nothing,Dict{Any,Any}(Pair{Any,Any}(:is_colorized,false),Pair{Any,Any}(:uppercase,false)))

In [4]:
#run(`tail semhuff_ipynb.log`)

In [5]:
import BlossomV: Matching, add_edge, get_match,PerfectMatchingCtx,solve

dense_num_edges(node_num) =  node_num*(node_num-1)÷2
Matching(node_num::Integer) = Matching(node_num,dense_num_edges)      
Matching(node_num::Integer, edge_num_max::Integer) = Matching(Int32(node_num), Int32(edge_num_max))

add_edge(matching::PerfectMatchingCtx,
        first_node::Integer,
        second_node::Integer, cost::Integer) = add_edge(matching, 
                                                        Int32(first_node), 
                                                        Int32(second_node),
                                                        Int32(cost))


function safe_add_edge(matching::PerfectMatchingCtx,
                        first_node::Integer,
                        second_node::Integer, cost::Integer)
    first_node != second_node || error("Can not have an edge between $(first_node) and itself")
    first_node >= 0  || error("first_node less than zero (value: $(first_node)). Indexes are zero-based")
    second_node >= 0  || error("second_node less than zero (value: $(second_node)). Indexes are zero-based")
    cost >= 0  || error("Cost must be positive. edge between $(first_node) and $(second_node) is $cost")
    
    debug("BlossomV: add_edge($matching, $(first_node), $(second_node), $cost)")
    add_edge(matching, first_node, second_node, cost)
end

get_match(matching::PerfectMatchingCtx, node::Integer) = get_match(matching, Int32(node))


function get_all_matchs(m::PerfectMatchingCtx, n_nodes::Integer) #HACK, you have to specidfiy the number of nodes, cos I can't pull it from the Matching object
    ret = Matrix{Int32}(2, n_nodes÷2)
    assigned = falses(n_nodes)
    kk = 1
    for ii in 0:n_nodes-1
        assigned[ii+1] && continue
        jj = get_match(m, ii)
        @assert(!assigned[jj+1])
        assigned[ii+1] = true
        assigned[jj+1] = true
        
        ret[1,kk] = ii
        ret[2,kk] = jj
        kk+=1
    end
    @assert(kk-1 == n_nodes÷2)
    return ret
end

get_all_matchs (generic function with 1 method)

In [21]:
"""
dist_func must resturn a values between 0.0 and 1.0. 
result is a matrix where each column is a pairing of items, expressed as there index (1 indexed)
"""
function most_similar_pairings(dist_func::Function, items, consider_nearest_n::Integer)
    m = Matching(length(items), min(dense_num_edges(length(items)), length(items)*consider_nearest_n))
    #sims = Matrix{Int32}(length(items),length(items)).*Inf #for Debug purposes
    for ii in (1:length(items)-1)
        jjs = ii+1:length(items)
        dists = [dist_func(items[ii], items[jj]) for jj in jjs]
        nearest_jjs = if consider_nearest_n < length(jjs)
            jjs[selectperm(dists, 1:consider_nearest_n)]
        else
            jjs
        end
            
        for (jj,dist) in zip(nearest_jjs,dists)
            @assert dist<=1.0
            scale = typemax(Int32)>>4
            approx_dist = round(Int32, dist*scale)
            #sims[ii,jj]=approx_dist
            #println(join([m, ii-1, jj-1, approx_dist],"\t"))
            safe_add_edge(m, ii-1,jj-1 , approx_dist)
        end
    end
    solve(m)
    get_all_matchs(m, length(items)) .+ 1    
end

most_similar_pairings

In [55]:
methods(BallTree)

# 4 methods for generic function "(::Type)":
(::Type{NearestNeighbors.BallTree}){T<:AbstractFloat,M<:Distances.Metric}(data::Array{T,2}, hyper_spheres::Array{NearestNeighbors.HyperSphere{T},1}, indices::Array{Int64,1}, metric::M, tree_data::NearestNeighbors.TreeData, reordered::Bool) at /home/ubuntu/.julia/v0.5/NearestNeighbors/src/ball_tree.jl:7
(::Type{NearestNeighbors.BallTree}){T<:AbstractFloat}(data::Array{T,2}) at /home/ubuntu/.julia/v0.5/NearestNeighbors/src/ball_tree.jl:42
(::Type{NearestNeighbors.BallTree}){T<:AbstractFloat,M<:Distances.Metric}(data::Array{T,2}, metric::M; leafsize, reorder, storedata, indicesfor, reorderbuffer) at /home/ubuntu/.julia/v0.5/NearestNeighbors/src/ball_tree.jl:42
(::Type{T}){T}(arg) at sysimg.jl:53

In [62]:
"""
dist_func must resturn a values between 0.0 and 1.0. 
result is a matrix where each column is a pairing of items, expressed as there index (1 indexed)
"""
function most_similar_pairings2(dist_func::Function, items, consider_nearest_n::Integer)
    
    m = Matching(length(items), min(dense_num_edges(length(items)), length(items)*consider_nearest_n))
    btree = BallTree(hcat(items...), AngularDist(); leafsize = 5) #TODO Use the dist_fun
    
    for ii in (1:length(items)-1)
        nearest_jjs, dists = knn(btree,items[ii], consider_nearest_n+1,false) #add 1 cos gonna remove self match
            
        for (jj,dist) in zip(nearest_jjs,dists)
            jj == ii && continue 
            
            @assert dist<=1.0
            scale = typemax(Int32)>>4
            approx_dist = round(Int32, dist*scale)
            #sims[ii,jj]=approx_dist
            #println(join([m, ii-1, jj-1, approx_dist],"\t"))
            safe_add_edge(m, ii-1,jj-1 , approx_dist)
        end
    end
    solve(m)
    get_all_matchs(m, length(items)) .+ 1    
end

most_similar_pairings2

In [57]:
eg_embeddings = [randn(50) for ii in 1:1_000];

In [58]:
gc()
timedata1 = @timed most_similar_pairings(angular_dist, eg_embeddings, 30)
gc()
timedata2 = @timed most_similar_pairings(angular_dist, eg_embeddings, 30)
println("-"^30)
println(timedata1[2:end])
println(timedata2[2:end])

perfect matching with 1000 nodes and 29535 edges
    starting init...done [0.006 secs]. 2 trees
    .0.
done [0.007 secs]. 152 grows, 0 expands, 11 shrinks
    expands: [0.000 secs], shrinks: [0.000 secs], dual updates: [0.000 secs]
perfect matching with 1000 nodes and 29535 edges
    starting init...done [0.006 secs]. 2 trees
    .0.
done [0.007 secs]. 152 grows, 0 expands, 11 shrinks
    expands: [0.000 secs], shrinks: [0.000 secs], dual updates: [0.000 secs]
------------------------------
(3.138574875,243893408,0.198454563,Base.GC_Diff(243893408,0,0,4715478,1512,0,198454563,10,0))
(3.099048185,239512432,0.202422361,Base.GC_Diff(239512432,0,0,4624225,1512,0,202422361,10,0))


In [63]:
gc()
timedata1 = @timed most_similar_pairings2(angular_dist, eg_embeddings, 30)
gc()
timedata2 = @timed most_similar_pairings2(angular_dist, eg_embeddings, 30)
println("-"^30)
println(timedata1[2:end])
println(timedata2[2:end])

LoadError: LoadError: InterruptException:
while loading In[63], in expression starting on line 211

In [7]:
"Returns a new tree, with the same structure but different values for the data"
function treemap(node::BranchNode; leaf_transform=identity, internal_transform=identity)
    function treemap_inner(node, leaf_transform, internal_transform)
        data = Trees.isleaf(node) ? leaf_transform(node.data) : internal_transform(node.data)
        children = BranchNode[treemap_inner(child, leaf_transform, internal_transform)  for child in node.children]
        new_node = BranchNode(copy(children), data)
    end
    treemap_inner(node, leaf_transform, internal_transform)
end
    

treemap

In [10]:
#TODO make this a testcase
x = BranchNode(
[BranchNode([],"11"),BranchNode([
        BranchNode([],"121"),BranchNode([],"122"),
        ],"12")],
    "1"
)
print_tree(STDOUT, x)

println()
y=treemap(x, leaf_transform = word->"L"*word, internal_transform = dummy -> "x"*dummy)
print_tree(STDOUT, y)

Trees.BranchNode with 2 children. data = "1"
├─ Trees.BranchNode with 0 children. data = "11"
└─ Trees.BranchNode with 2 children. data = "12"
   ├─ Trees.BranchNode with 0 children. data = "121"
   └─ Trees.BranchNode with 0 children. data = "122"

Trees.BranchNode with 2 children. data = "x1"
├─ Trees.BranchNode with 0 children. data = "L11"
└─ Trees.BranchNode with 2 children. data = "x12"
   ├─ Trees.BranchNode with 0 children. data = "L121"
   └─ Trees.BranchNode with 0 children. data = "L122"


In [15]:
"""
Assumes that the `tree` is already a Huffman tree.
"""
function semhuff(classification_tree, embeddings, consider_nearest_n)
    embedding_dim = length(first(embeddings))
    sim_tree = treemap(classification_tree, 
                            leaf_transform = word->word,
                            internal_transform = dummy -> "")
    
    nodes_at_depth, codes_at_depth = levels(sim_tree)
    maxdepth = length(nodes_at_depth)
    
    
    embeds = [embeddings[node.data] for node in nodes_at_depth[maxdepth]]
    #Dict(code => embeddings[node.data] for (code, node) in zip(codes_at_depth[maxdepth], nodes_at_depth[maxdepth]))
    for depth in maxdepth:-1:2
        info("semantically sorting level: $depth")
        nodes = nodes_at_depth[depth]
        
        pair_indexes = most_similar_pairings(Query.angular_dist, embeds, consider_nearest_n)
        
        #We will now, assign the new nodes to parents in arbitary order
        nodes_above = nodes_at_depth[depth-1]
        embeds_above = typeof(embeds)(length(nodes_above))
        pair_jj = 1

        for (above_ii,node_above) in enumerate(nodes_above)

            if Trees.isleaf(node_above)
                embeds_above[above_ii] = embeddings[node_above.data]
            else
                #It is a branch so put a pair of nodes here
                @assert (length(node_above.children) == 2)
                child_index1  = pair_indexes[1,pair_jj]
                child_index2  = pair_indexes[2,pair_jj]
                pair_jj += 1
                node_above.children[1] = nodes[child_index1]
                node_above.children[2] = nodes[child_index2]
                embeds_above[above_ii] = (embeds[child_index1] + embeds[child_index2])/2.0
                #@show nodes[child_index1], nodes[child_index2]
            end
        end
        @assert(pair_jj - 1  == size(pair_indexes,2), "$(pair_jj) != $(length(pair_indexes)) + 1") #All pairs must be assigned.
            
        embeds = embeds_above
    end
    sim_tree
end
    
    

semhuff

In [19]:
midi_tree = ee.classification_tree
@time semtree = semhuff(midi_tree, ee.embedding, 30);
#print_tree(STDOUT, semtree)

2016-07-28T18:52:51.549 - info: semantically sorting level: 22
perfect matching with 6518 nodes and 195075 edges
    starting init...done [0.156 secs]. 10 trees
    .8.6.4.2.0.
done [0.174 secs]. 928 grows, 0 expands, 17 shrinks
    expands: [0.000 secs], shrinks: [0.000 secs], dual updates: [0.000 secs]
2016-07-28T18:53:19.696 - info: semantically sorting level: 21
perfect matching with 32324 nodes and 969255 edges
    starting init...done [2.009 secs]. 68 trees
    .64.56.48.40.32.24.16.8.6.4.2.0.
done [2.358 secs]. 7170 grows, 4 expands, 99 shrinks
    expands: [0.001 secs], shrinks: [0.002 secs], dual updates: [0.001 secs]
2016-07-28T18:59:08.737 - info: semantically sorting level: 20
perfect matching with 34866 nodes and 1045515 edges
    starting init...done [2.114 secs]. 78 trees
    .64.56.48.40.32.24.16.8.6.4.2.0.
done [2.313 secs]. 6635 grows, 4 expands, 117 shrinks
    expands: [0.001 secs], shrinks: [0.003 secs], dual updates: [0.001 secs]
2016-07-28T19:06:01.238 - info: se

""

In [18]:
1175.463271  / 60

19.591054516666667

In [17]:
open("tree.txt","w") do fp
    print_tree(fp, semtree)
end

In [ ]:
ee_keys = rand(1:length(ee.embedding),6)
wvs = collect(values(ee.embedding))[ee_keys]
words = collect(keys(ee.embedding))[ee_keys]

In [ ]:
using Query
wvs = [randn(50) for i in 1:6]
m=Matching(Int32(length(wvs)),Int32(length(wvs))^2)

In [ ]:
sims = Matrix{Int32}(length(wvs),length(wvs)).*Inf
for ii::Int32 in (1:length(wvs)-1)
    wv_ii=wvs[ii]
    for jj::Int32 in (ii+1:length(wvs))
        
        wv_jj=wvs[jj]
        dist = Query.angular_dist(wv_ii, wv_jj)
        
        scale = typemax(Int32)>>16
        approx_dist = round(Int32, dist*scale)
        sims[ii,jj]=approx_dist
        println(join([m, ii-1, jj-1, approx_dist],"\t"))
        add_edge(m, ii-1,jj-1 , approx_dist)
    end
end
sims

In [ ]:
BlossomV.solve(m)

In [ ]:
for ii in (1:6)-1
    match = get_match(m,ii)+1
    greed = indmin(sims[ii+1,:]-1)
    println(ii+1,"\t|\t", match, "\t", greed)
end

In [ ]:

collect(get_all_matchs(m,6))

In [ ]:
Base.getindex(node::BranchNode, idx) = Base.getindex(node.children, idx)


In [ ]:
@inline function Base.exp2(x::Int64)
    if x > 1023 
        Inf 
    elseif x < -1074
        0.0
    else 
        reinterpret(Float64,x > -1023 ? ((1<<62)+(x-1)<<52) : 1<<(x+1074))
    end
end

function get_path_dist(code1,code2)
    l1 = length(code1)
    l2 = length(code2)
    lc = 0 #Common length
    for lc in 1:min(l1,l2)
        if code1[lc]!=code2[lc]
            lc-=1 #Last didn't match
            break
        end
    end
    ld1 = l1-lc
    ld2 = l2-lc
    ld1+ld2
end

get_path_sim(code1,code2) = exp2(-get_path_dist(code1,code2))

using Base.Test
@test get_path_sim([1],[1]) == 1.0
@test get_path_sim([1],[0]) == 0.25

@test get_path_sim([1,1],[1,0]) == 0.25
@test get_path_sim([1,1],[1,1]) == 1.0
@test get_path_sim([1,1],[0,1]) == 1/16
@test get_path_sim([1,1],[0,0]) == 1/16

@test get_path_sim([1],[1,1]) == 0.5
@test get_path_sim([1],[1,1,1]) == 0.25
@test get_path_sim([1,1],[1,1,1]) == 0.5
@test get_path_sim([1,0],[1,1,1]) == 0.125
@test get_path_sim([1,1,1],[1,1,1]) == 1.0


In [ ]:
ee 